# Wildfire Dynamics, Atmospheric Emissions and Forest Cover Loss
## Zambia - Tanzania - Malawi - Mozambique  (2015-2024)
### A Decadal Multi-Source Remote Sensing Analysis

> **Author:** Ujjwal Kumar Swain  
> **Data:** NASA MODIS MCD64A1 + Copernicus Sentinel-5P TROPOMI + Hansen GFC v1.11  
> **Platform:** Google Earth Engine (Python API) + Google Colab

**Research context:**  
Southern and eastern Africa host some of the world's most fire-prone savanna and miombo woodland ecosystems.
Zambia and Mozambique consistently rank among the top 5 globally for burned area extent.
This notebook brings together three independent satellite datasets to characterise fire seasonality,
landscape-level recurrence, associated atmospheric emissions, and the relationship between
fire regimes and forest cover loss over a 10-year period.

**Notebook structure:**
1. Environment setup  
2. Study area & temporal parameters  
3. MODIS burned area extraction (GEE)  
4. Seasonal analysis  
5. Visualisation -- Figures 1-4  
6. Google Drive mount + raster ingestion  
7. Country-wise burned area (GEE)  
8. Sentinel-5P atmospheric analysis (GEE)  
9. Hansen GFC forest cover loss (GEE)  
10. Interactive Folium map  
11. Summary report  
12. Export

## Section 1: Environment Setup

In [ ]:
!pip install -q geemap earthengine-api geopandas rasterio rasterstats contextily folium matplotlib-scalebar

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-ujjwalkumarswainiirs1')
print('GEE initialised successfully')

In [ ]:
import warnings
import os

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
import seaborn as sns
import folium
from folium.plugins import HeatMap, MiniMap, Fullscreen, MousePosition, MeasureControl
from IPython.display import HTML
import geemap

warnings.filterwarnings('ignore')

# Create output directory structure
for d in ['outputs/figures', 'outputs/maps', 'data/processed', 'data/raw']:
    os.makedirs(d, exist_ok=True)

print('Libraries imported — directory structure ready')


## Section 2: Study Area and Time Period

In [ ]:
# Bounding box: Zambia, Tanzania, Malawi, Mozambique
AOI_COORDS     = [25, -20, 40, -5]   # [lon_min, lat_min, lon_max, lat_max]
aoi            = ee.Geometry.Rectangle(AOI_COORDS)

START_DATE     = '2015-01-01'
END_DATE       = '2024-12-31'
YEARS          = list(range(2015, 2025))
COUNTRIES_LIST = ['Zambia', 'Tanzania', 'Malawi', 'Mozambique']

# Month labels used throughout
MONTH_LABELS = ['Jan','Feb','Mar','Apr','May','Jun',
                'Jul','Aug','Sep','Oct','Nov','Dec']

print(f'Study countries : {", ".join(COUNTRIES_LIST)}')
print(f'Bounding box    : lon {AOI_COORDS[0]}°–{AOI_COORDS[2]}°E  '
      f'lat {AOI_COORDS[1]}°–{AOI_COORDS[3]}°')
print(f'Temporal range  : {START_DATE} to {END_DATE}  ({len(YEARS)} years)')


## Section 3: Data Acquisition

### 3.1.  Monthly Burned Area Time Series (MODIS MCD64A1)

MODIS MCD64A1 provides monthly burned area at 500 m derived from the active-fire
signal and surface reflectance change.  The `BurnDate` band stores the Julian day
of burning; any pixel with BurnDate > 0 was confirmed burned that month.
We sum pixel areas across the entire AOI to get a monthly km2 time series.

In [ ]:
burned_collection = (
    ee.ImageCollection('MODIS/061/MCD64A1')
    .filterDate(START_DATE, END_DATE)
    .filterBounds(aoi)
    .select('BurnDate')
)


def get_burned_area_km2(image):
    """Compute burned area (km²) for a single MODIS monthly image."""
    burned_mask    = image.gt(0)                      # BurnDate > 0 → burned pixel
    pixel_area_km2 = ee.Image.pixelArea().divide(1e6)
    burned_area    = (
        burned_mask
        .multiply(pixel_area_km2)
        .reduceRegion(
            reducer    = ee.Reducer.sum(),
            geometry   = aoi,
            scale      = 500,
            maxPixels  = 1e13,
            bestEffort = True,
        )
    )
    return image.set({
        'burned_area_km2'    : burned_area.get('BurnDate'),
        'system:time_start'  : image.get('system:time_start'),
    })


print('Extracting burned area statistics — ~2-4 minutes...')

burned_with_stats = burned_collection.map(get_burned_area_km2)
time_series       = burned_with_stats.aggregate_array('burned_area_km2').getInfo()
timestamps        = burned_with_stats.aggregate_array('system:time_start').getInfo()

df_burned = (
    pd.DataFrame({'timestamp_ms': timestamps, 'burned_area_km2': time_series})
    .assign(
        date          = lambda d: pd.to_datetime(d['timestamp_ms'], unit='ms'),
        year          = lambda d: d['date'].dt.year,
        month         = lambda d: d['date'].dt.month,
        burned_area_km2 = lambda d: pd.to_numeric(d['burned_area_km2'],
                                                   errors='coerce').fillna(0),
    )
    .sort_values('date')
    .reset_index(drop=True)
)

df_burned.to_csv('data/processed/monthly_burned_area_2015_2024.csv', index=False)

print(f'Extracted {len(df_burned)} monthly records')
print(f'10-year total burned area: {df_burned["burned_area_km2"].sum():,.0f} km²')
df_burned.head(6)


### 3.2.  Fire Frequency and Land Cover Rasters --> Google Drive

The fire frequency raster counts how many months each 500 m pixel burned across
the full 120-month record -- essentially a recurrence index.  We also need the
MODIS IGBP land cover (MCD12Q1 Type 1) to stratify fire behaviour by vegetation type.

Both exports go to Google Drive / EarthEngine folder.  While they run (~5-10 min)
you can continue to Section 4 which only needs the CSV.

In [ ]:
# ---- Fire frequency raster: count of months burned per pixel --------------
fire_frequency = (
    burned_collection
    .map(lambda img: img.gt(0).rename('burned'))
    .sum()
    .toFloat()                  # GEE requires float for Drive export
    .clip(aoi)
    .rename('fire_frequency')
)

task_ff = ee.batch.Export.image.toDrive(
    image          = fire_frequency,
    description    = 'fire_frequency_2015_2024',
    folder         = 'EarthEngine',
    fileNamePrefix = 'fire_frequency_2015_2024',
    region         = aoi,
    scale          = 500,
    crs            = 'EPSG:4326',
    maxPixels      = 1e13,
)
task_ff.start()

# ---- MODIS land cover 2020 (IGBP Type 1) --------------------------------
# Land cover: single MCD12Q1 scene from 2020 (mid-period representative year).
# LC_Type1 is the IGBP classification -- 17 classes, widely used in fire ecology.
land_cover = (
    ee.ImageCollection('MODIS/061/MCD12Q1')
    .filterDate('2020-01-01', '2020-12-31')
    .first()
    .select('LC_Type1')
    .clip(aoi)
)

task_lc = ee.batch.Export.image.toDrive(
    image          = land_cover,
    description    = 'landcover_MODIS_2020',
    folder         = 'EarthEngine',
    fileNamePrefix = 'landcover_MODIS_2020',
    region         = aoi,
    scale          = 500,
    crs            = 'EPSG:4326',
    maxPixels      = 1e13,
)
task_lc.start()

print(f'Fire frequency export : {task_ff.status()["state"]}')
print(f'Land cover export     : {task_lc.status()["state"]}')
print('Monitor tasks at: https://code.earthengine.google.com/tasks')
print('Continue to Section 4 while exports complete (~5-10 min).')


## Section 4: Seasonal Analysis

In [ ]:
df = (
    pd.read_csv('data/processed/monthly_burned_area_2015_2024.csv',
                parse_dates=['date'])
    .assign(
        year  = lambda d: d['date'].dt.year,
        month = lambda d: d['date'].dt.month,
    )
)

# Monthly climatology- mean and SD across 10 years
# Monthly climatology -- average and spread across the 10 years.
# The SD matters here: wide SD in Aug-Sep means high inter-annual variability,
# which is expected given ENSO-driven rainfall anomalies in this region.
monthly_clim = (
    df.groupby('month')['burned_area_km2']
    .agg(mean='mean', std='std', total='sum')
    .reset_index()
    .assign(month_label = lambda d: d['month'].map(lambda x: MONTH_LABELS[x-1]))
)

# Annual totals and heatmap pivot
annual_totals = df.groupby('year')['burned_area_km2'].sum()
pivot         = df.pivot_table(index='year', columns='month',
                               values='burned_area_km2', aggfunc='sum')
pivot.columns = MONTH_LABELS

peak_mo   = monthly_clim.loc[monthly_clim['mean'].idxmax(), 'month_label']
peak_area = monthly_clim['mean'].max()

print('=== FIRE SEASONALITY STATISTICS ===')
print(f'Peak month  : {peak_mo} ({peak_area:,.0f} km²)')
print(f'10-yr total : {annual_totals.sum():,.0f} km²')
print(f'Annual mean : {annual_totals.mean():,.0f} km²/yr')
print(f'Peak year   : {annual_totals.idxmax()} ({annual_totals.max():,.0f} km²)')
print(f'Min year    : {annual_totals.idxmin()} ({annual_totals.min():,.0f} km²)')


## Section 5: Visualisation (Figures 1-4)

### Figure 1- Monthly Fire Calendar Heatmap + Annual Totals

The heatmap is probably the most useful single figure for quickly reading
fire seasonality: you can spot the Jul-Oct compression at a glance, and the
row-by-row variation shows which years were anomalously active or quiet.

In [ ]:
plt.rcParams.update({'font.family': 'DejaVu Sans', 'font.size': 11})

fig, axes = plt.subplots(
    2, 1, figsize=(14, 11),
    gridspec_kw={'height_ratios': [2.5, 1], 'hspace': 0.45},
)

# ── Panel 1: Heatmap ─────────────────────────────────────────────────────
ax1 = axes[0]
sns.heatmap(
    pivot, cmap='YlOrRd', linewidths=0.8, linecolor='#f0f0f0',
    annot=True, fmt='.0f', annot_kws={'size': 8.5},
    ax=ax1, cbar_kws={'label': 'Burned Area (km²)', 'shrink': 0.8},
)
ax1.set_title(
    'Monthly Burned Area — Zambia, Tanzania, Malawi & Mozambique (2015–2024)\n'
    'MODIS MCD64A1 | 500m | Values in km²',
    fontsize=13, fontweight='bold', pad=14,
)
ax1.set_xlabel('Month', fontsize=11)
ax1.set_ylabel('Year',  fontsize=11)
ax1.tick_params(axis='x', rotation=0)
ax1.tick_params(axis='y', rotation=0)
ax1.axvline(x=6,  color='#2166ac', linewidth=2, linestyle='--', alpha=0.6)
ax1.axvline(x=10, color='#2166ac', linewidth=2, linestyle='--', alpha=0.6)
ax1.text(7.9, -0.55, 'Peak fire season (Jul–Oct)',
         color='#2166ac', fontsize=9, ha='center')

# ── Panel 2: Annual bar chart ─────────────────────────────────────────────
ax2        = axes[1]
bar_colors = ['#d73027' if v > annual_totals.mean() else '#fc8d59'
              for v in annual_totals.values]
bars = ax2.bar(
    annual_totals.index, annual_totals.values,
    color=bar_colors, edgecolor='white', linewidth=0.8, zorder=3,
)
ax2.axhline(
    annual_totals.mean(), color='#4d4d4d', linestyle='--',
    linewidth=1.2, zorder=4,
    label=f'10-yr mean: {annual_totals.mean():,.0f} km²',
)
for bar, val in zip(bars, annual_totals.values):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + annual_totals.max() * 0.01,
        f'{val/1e3:.0f}k', ha='center', va='bottom', fontsize=7.5,
    )
ax2.set_title('Annual Total Burned Area — All 4 Countries (km²)',
              fontsize=12, fontweight='bold', pad=10)
ax2.set_xlabel('Year', fontsize=10)
ax2.set_ylabel('km²',  fontsize=10)
ax2.set_xticks(annual_totals.index)
ax2.tick_params(axis='x', rotation=45)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1e3:.0f}k'))
ax2.legend(fontsize=9)
ax2.grid(axis='y', alpha=0.3, zorder=1)
ax2.set_facecolor('#fafafa')

fig.text(
    0.01, 0.01,
    'Data: NASA MODIS MCD64A1 v061 via GEE | '
    'Study area: Zambia + Tanzania + Malawi + Mozambique',
    fontsize=8, color='#666', style='italic',
)
plt.savefig('outputs/figures/fig1_seasonal_heatmap.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 1 saved')


### Figure 2: Fire Seasonality Climatology

Plotting individual year traces as semi-transparent lines behind the mean bars
gives a sense of how tightly the fire season is constrained -- if most years
cluster together the season is predictable; wide spread indicates ENSO sensitivity.

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))

# Background: individual year traces
for yr in YEARS:
    yr_data = df[df['year'] == yr].sort_values('month')
    if len(yr_data) == 12:
        ax.plot(yr_data['month'], yr_data['burned_area_km2'],
                color='#fc8d59', alpha=0.25, linewidth=1.0, zorder=2)

# Bar colours by season
bar_colors = [
    '#d73027' if m in [7, 8, 9, 10]
    else '#fdae61' if m in [6, 11]
    else '#e0e0e0'
    for m in range(1, 13)
]
ax.bar(monthly_clim['month'], monthly_clim['mean'],
       color=bar_colors, edgecolor='white', alpha=0.7, zorder=3,
       label='Monthly mean (2015–2024)')
ax.errorbar(monthly_clim['month'], monthly_clim['mean'],
            yerr=monthly_clim['std'],
            fmt='none', color='#333', capsize=5, capthick=1.5,
            linewidth=1.5, zorder=5)
ax.plot(monthly_clim['month'], monthly_clim['mean'],
        color='#1a1a1a', linewidth=2.5, marker='o', markersize=6,
        zorder=6, label='Mean trend')
ax.axvspan(6.5, 10.5, alpha=0.08, color='red', zorder=1,
           label='Peak fire season (Jul–Oct)')

# Peak annotation
pk_idx = monthly_clim['mean'].idxmax()
ax.annotate(
    f"Peak: {monthly_clim.loc[pk_idx, 'mean']:,.0f} km²",
    xy     =(monthly_clim.loc[pk_idx, 'month'],
             monthly_clim.loc[pk_idx, 'mean']),
    xytext =(monthly_clim.loc[pk_idx, 'month'] + 0.6,
             monthly_clim.loc[pk_idx, 'mean'] * 1.06),
    fontsize=10, color='#d73027', fontweight='bold',
    arrowprops=dict(arrowstyle='->', color='#d73027', lw=1.5),
)

ax.set_xticks(range(1, 13))
ax.set_xticklabels(MONTH_LABELS, fontsize=11)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1e3:.0f}k'))
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Burned Area (km²)', fontsize=12)
ax.set_title(
    'Fire Seasonality Climatology — Zambia, Tanzania, Malawi & Mozambique (2015–2024)\n'
    'Mean monthly burned area ±1 SD | MODIS MCD64A1',
    fontsize=13, fontweight='bold', pad=12,
)
ax.legend(fontsize=10)
ax.set_facecolor('#fafafa')
ax.grid(axis='y', alpha=0.3)
ax.spines[['top', 'right']].set_visible(False)

plt.tight_layout()
plt.savefig('outputs/figures/fig2_fire_climatology.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 2 saved')


## Section 6: Google Drive Mount and Raster Ingestion

GEE exports should be done by now (~5-10 min from Section 3.2).
Mount Drive and copy the TIFs locally so rasterio can read them.

In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

DRIVE_FOLDER = '/content/drive/MyDrive/EarthEngine'

for src, dst in [
    ('fire_frequency_2015_2024.tif', 'data/raw/fire_frequency_2015_2024.tif'),
    ('landcover_MODIS_2020.tif',     'data/raw/landcover_MODIS_2020.tif'),
]:
    full_src = f'{DRIVE_FOLDER}/{src}'
    if os.path.exists(full_src):
        shutil.copy(full_src, dst)
        print(f'  Copied: {src}')
    else:
        print(f'  Not found: {full_src}')
        print('  → Check GEE Tasks: https://code.earthengine.google.com/tasks')


### Figure 3: Fire Frequency by Land Cover Type

In [ ]:
# MODIS IGBP classification -- 17 land cover classes.
# In this region the ecologically important ones are:
# Woody Savanna (8), Savanna (9), Deciduous Broadleaf Forest (4) -- miombo woodlands,
# and Cropland (12) / Cropland-Vegetation Mosaic (14) for agricultural landscapes.
IGBP_CLASSES = {
     1: 'Evergreen Needleleaf Forest',  2: 'Evergreen Broadleaf Forest',
     3: 'Deciduous Needleleaf Forest',  4: 'Deciduous Broadleaf Forest',
     5: 'Mixed Forest',                 6: 'Closed Shrubland',
     7: 'Open Shrubland',               8: 'Woody Savanna',
     9: 'Savanna',                     10: 'Grassland',
    11: 'Permanent Wetland',           12: 'Cropland',
    13: 'Urban & Built-up',            14: 'Cropland/Veg Mosaic',
    15: 'Snow & Ice',                  16: 'Barren',
    17: 'Water Bodies',
}

IGBP_COLORS = {
    'Evergreen Broadleaf Forest': '#1a6b1a',
    'Deciduous Broadleaf Forest': '#52b152',
    'Mixed Forest'              : '#7fbf7b',
    'Woody Savanna'             : '#d4a017',
    'Savanna'                   : '#e8c14a',
    'Grassland'                 : '#a8d468',
    'Cropland'                  : '#e07b54',
    'Cropland/Veg Mosaic'       : '#c45a38',
    'Open Shrubland'            : '#bfad8c',
    'Closed Shrubland'          : '#9c8c6a',
    'Permanent Wetland'         : '#4d9fbf',
    'Urban & Built-up'          : '#888888',
    'Water Bodies'              : '#2b6fa8',
    'Barren'                    : '#d9d9d9',
}

# ------ Load rasters and cross-tabulate ------------------------------
# Load rasters -- resample LC to match fire frequency grid (nearest neighbour,
# since LC is categorical)
with rasterio.open('data/raw/fire_frequency_2015_2024.tif') as src:
    fire_arr       = src.read(1).astype(float)
    fire_shape     = fire_arr.shape
    fire_transform = src.transform

with rasterio.open('data/raw/landcover_MODIS_2020.tif') as src:
    lc_arr = src.read(1, out_shape=fire_shape,
                      resampling=Resampling.nearest).astype(float)

# Flatten and filter valid pixels
fire_flat = fire_arr.flatten()
lc_flat   = lc_arr.flatten()
valid     = (fire_flat >= 0) & (lc_flat > 0) & (lc_flat < 255)

df_spatial = (
    pd.DataFrame({'fire_count': fire_flat[valid], 'lc_class': lc_flat[valid]})
    .assign(lc_label = lambda d: d['lc_class'].map(IGBP_CLASSES))
    .dropna(subset=['lc_label'])
)

lc_summary = (
    df_spatial.groupby('lc_label')
    .agg(
        mean_fire_freq     = ('fire_count', 'mean'),
        total_fire_events  = ('fire_count', 'sum'),
        pixel_count        = ('fire_count', 'count'),
        pct_burned         = ('fire_count', lambda x: (x > 0).mean() * 100),
    )
    .reset_index()
    .query('pixel_count > 100')
    .sort_values('mean_fire_freq')
)
lc_summary.to_csv('data/processed/fire_by_landcover.csv', index=False)

print(lc_summary[['lc_label', 'mean_fire_freq', 'pct_burned', 'pixel_count']].to_string(index=False))


In [ ]:
# The dual-panel layout shows two complementary things:
# Panel A: how intensively each class burns (mean recurrence frequency)
# Panel B: what proportion of each class has burned at all
# Together they tell us which ecosystems are pyrogenic vs. fire-resistant.

bar_colors_lc = [IGBP_COLORS.get(lbl, '#999') for lbl in lc_summary['lc_label']]
overall_mean  = lc_summary['mean_fire_freq'].mean()

fig, (ax_l, ax_r) = plt.subplots(
    1, 2, figsize=(16, 6),
    gridspec_kw={'wspace': 0.4},
)

# ---- Panel A: Mean fire frequency ----------------------------------------------
bars = ax_l.barh(lc_summary['lc_label'], lc_summary['mean_fire_freq'],
                 color=bar_colors_lc, edgecolor='white', height=0.65)
ax_l.axvline(overall_mean, color='#333', linestyle='--',
             linewidth=1.3, label=f'Overall mean: {overall_mean:.1f}')
for bar, val in zip(bars, lc_summary['mean_fire_freq']):
    ax_l.text(val + 0.05, bar.get_y() + bar.get_height() / 2,
              f'{val:.1f}', va='center', ha='left', fontsize=9)
ax_l.set_xlabel('Mean Fire Frequency (months burned per pixel, 2015–2024)', fontsize=10)
ax_l.set_title('A. Mean Fire Frequency by Land Cover', fontsize=12, fontweight='bold')
ax_l.legend(fontsize=9)
ax_l.grid(axis='x', alpha=0.3)
ax_l.spines[['top', 'right']].set_visible(False)
ax_l.set_facecolor('#fafafa')

# ---- Panel B: Proportion ever burned ----------------------------------------
bars2 = ax_r.barh(lc_summary['lc_label'], lc_summary['pct_burned'],
                  color=bar_colors_lc, edgecolor='white', height=0.65)
for bar, val in zip(bars2, lc_summary['pct_burned']):
    ax_r.text(val + 0.5, bar.get_y() + bar.get_height() / 2,
              f'{val:.0f}%', va='center', ha='left', fontsize=9)
ax_r.set_xlabel('% of Pixels Burned at Least Once (2015–2024)', fontsize=10)
ax_r.set_title('B. Proportion of Land Cover Area Ever Burned', fontsize=12, fontweight='bold')
ax_r.set_xlim(0, 105)
ax_r.grid(axis='x', alpha=0.3)
ax_r.spines[['top', 'right']].set_visible(False)
ax_r.set_facecolor('#fafafa')

fig.suptitle(
    'Wildfire Dynamics Across Land Cover Types\n'
    'Zambia, Tanzania, Malawi & Mozambique | MODIS MCD64A1 × MCD12Q1 | 2015–2024',
    fontsize=13, fontweight='bold', y=1.02,
)
plt.savefig('outputs/figures/fig3_fire_by_landcover.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 3 saved')


### Figure 4: Spatial Fire Recurrence Map

In [ ]:
import contextily as ctx

fig, ax = plt.subplots(1, 1, figsize=(11, 12))
# Re-read the fire frequency raster for mapping
# (we already have fire_arr in memory from the land cover analysis,
#  but reloading is cleaner in case this cell is run in isolation)

with rasterio.open('data/raw/fire_frequency_2015_2024.tif') as src:
    fire_data = src.read(1).astype(float)
    bounds    = src.bounds
    crs_str   = src.crs.to_string()

# Classify into 5 recurrence intervals
# Thresholds chosen to match roughly equal area classes while
# preserving ecological meaning -- "rare" = 1-2 months over 10 years,
# "extreme" = 9+ months (nearly every dry season).
classified = np.full_like(fire_data, np.nan)
classified[(fire_data >= 1) & (fire_data <= 2)] = 1
classified[(fire_data >= 3) & (fire_data <= 4)] = 2
classified[(fire_data >= 5) & (fire_data <= 6)] = 3
classified[(fire_data >= 7) & (fire_data <= 8)] = 4
classified[fire_data >= 9]                      = 5

RECURRENCE_COLORS = ['#fee391', '#fe9929', '#d95f0e', '#993404', '#4d0000']
RECURRENCE_LABELS = [
    'Rare (1–2 months)',         'Occasional (3–4 months)',
    'Frequent (5–6 months)',     'Very Frequent (7–8 months)',
    'Extreme (9+ months)',
]
cmap_cls = mcolors.ListedColormap(RECURRENCE_COLORS)
norm_cls = BoundaryNorm([0.5, 1.5, 2.5, 3.5, 4.5, 5.5], cmap_cls.N)

ax.set_xlim(bounds.left,  bounds.right)
ax.set_ylim(bounds.bottom, bounds.top)

try:
    ctx.add_basemap(ax, crs=crs_str,
                    source=ctx.providers.CartoDB.PositronNoLabels,
                    zoom=6, alpha=0.5)
except Exception as e:
    ax.set_facecolor('#d9d9d9')
    print(f'Basemap unavailable — using solid background: {e}')

extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]
ax.imshow(classified, extent=extent, origin='upper',
          cmap=cmap_cls, norm=norm_cls, alpha=0.92, aspect='auto', zorder=2)

# Country boundaries from Natural Earth
try:
    world  = gpd.read_file(
        'https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_admin_0_countries.zip')
    study4 = world[world['NAME'].isin(COUNTRIES_LIST)]
    nbrs   = world[world['NAME'].isin([
        'Zimbabwe', 'Angola', 'Kenya', 'Uganda',
        'Democratic Republic of the Congo',
        'Namibia', 'Botswana', 'South Africa',
    ])]
    study4.boundary.plot(ax=ax, linewidth=2.2, edgecolor='#1a1a1a', zorder=4)
    nbrs.boundary.plot(ax=ax, linewidth=0.8, edgecolor='#666',
                       linestyle='--', zorder=3)
except Exception as e:
    print(f'Boundary overlay failed: {e}')

for name, tx, ty in [
    ('ZAMBIA',     0.22, 0.40),
    ('TANZANIA',   0.60, 0.70),
    ('MALAWI',     0.64, 0.45),
    ('MOZAMBIQUE', 0.66, 0.28),
]:
    ax.text(tx, ty, name, transform=ax.transAxes, fontsize=10,
            color='white', fontweight='bold', zorder=5,
            bbox=dict(boxstyle='round,pad=0.25', facecolor='#1a1a1a', alpha=0.7))

legend_patches = [mpatches.Patch(color=c, label=l)
                  for c, l in zip(RECURRENCE_COLORS, RECURRENCE_LABELS)]
legend = ax.legend(
    handles=legend_patches,
    title='Fire Recurrence Class\n(months burned / 10 yrs)',
    title_fontsize=9, fontsize=9, loc='lower left',
    framealpha=0.92, edgecolor='#aaa', fancybox=True,
)
legend.get_title().set_fontweight('bold')

ax.set_title(
    '10-Year Fire Frequency\n'
    'Zambia, Tanzania, Malawi & Mozambique (2015–2024)\n'
    'MODIS MCD64A1 | 500m | Classified by recurrence interval',
    fontsize=12, fontweight='bold', pad=12,
)
ax.set_xlabel('Longitude', fontsize=10)
ax.set_ylabel('Latitude',  fontsize=10)
fig.text(0.01, 0.01,
         'Data: NASA MODIS MCD64A1 v061 | GEE | WGS84 | 4-country study area',
         fontsize=8, color='#555', style='italic')

plt.tight_layout()
plt.savefig('outputs/figures/fig4_spatial_fire_map.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 4 saved')


## Section 7: Country-wise Burned Area Extraction
We need per-country totals for the interactive map and cross-country comparisons.
LSIB (USDOS Large Scale International Boundaries) gives clean, consistent polygons
that match what GEE uses internally -- better than trying to filter GADM by country name.

In [ ]:
# Build country geometry dictionary from LSIB
lsib = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
country_geoms = {
    c: lsib.filter(ee.Filter.eq('country_na', c)).geometry()
    for c in COUNTRIES_LIST
}

print('Extracting per-country burned area — ~3-4 minutes...')

country_yearly = {c: {} for c in country_geoms}

for country, geom in country_geoms.items():
    for yr in YEARS:
        yr_col = burned_collection.filter(ee.Filter.calendarRange(yr, yr, 'year'))
        area   = (
            yr_col
            .map(lambda img: img.gt(0).multiply(ee.Image.pixelArea().divide(1e6)))
            .sum()
            .reduceRegion(
                reducer    = ee.Reducer.sum(),
                geometry   = geom,
                scale      = 500,
                maxPixels  = 1e13,
                bestEffort = True,
            )
        )
        val = area.getInfo().get('BurnDate', 0)
        country_yearly[country][yr] = round(float(val), 1) if val else 0.0
    print(f'  {country}: done')

df_country = pd.DataFrame([
    {'Country': c, 'Year': yr, 'Burned_km2': v}
    for c, yd in country_yearly.items()
    for yr, v in yd.items()
])
df_country.to_csv('data/processed/country_burned_area.csv', index=False)

pivot_c = df_country.pivot(index='Country', columns='Year', values='Burned_km2')
pivot_c['Total']   = pivot_c.sum(axis=1)
pivot_c['Mean/yr'] = (pivot_c['Total'] / 10).round(0)
print('\n=== COUNTRY-WISE BURNED AREA (km²) ===')
print(pivot_c.round(0).to_string())


## Section: Sentinel-5P Atmospheric Analysis

### 8.1.  GEE Extraction- CO, NO₂ and Absorbing Aerosol Index (2018-2024)

TROPOMI on Sentinel-5P launched in October 2017; the OFFL (offline, science-grade)
product is available from mid-2018 onward.  We extract three trace gas / aerosol
products that are all known to respond to biomass burning:

- **CO** (carbon monoxide) -- primary combustion tracer, long atmospheric lifetime
- **NO2** (tropospheric column) -- shorter-lived, useful for identifying active fire proximity
- **AAI** (absorbing aerosol index) -- smoke and dust proxy; positive values indicate
  absorbing aerosol above the UV-reflecting surface

Scale factors convert from native mol/m2 to more interpretable units.

In [ ]:
import time

S5P_YEARS = list(range(2018, 2025))   # TROPOMI available from mid-2018

S5P_PRODUCTS = {
    'CO': {
        'collection'  : 'COPERNICUS/S5P/OFFL/L3_CO',
        'band'        : 'CO_column_number_density',
        'scale_factor': 1e3,      # mol/m² → mmol/m²
        'unit'        : 'mmol/m²',
        'label'       : 'CO Column Density',
    },
    'NO2': {
        'collection'  : 'COPERNICUS/S5P/OFFL/L3_NO2',
        'band'        : 'tropospheric_NO2_column_number_density',
        'scale_factor': 1e6,      # mol/m² → µmol/m²
        'unit'        : 'µmol/m²',
        'label'       : 'NO₂ Tropospheric Column',
    },
    'AER_AI': {
        'collection'  : 'COPERNICUS/S5P/OFFL/L3_AER_AI',
        'band'        : 'absorbing_aerosol_index',
        'scale_factor': 1.0,      # dimensionless
        'unit'        : 'AAI',
        'label'       : 'Absorbing Aerosol Index',
    },
}

# Rebuild country_geoms if needed (e.g. if kernel restarted after Section 7)
if 'country_geoms' not in dir():
    lsib = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
    country_geoms = {
        c: lsib.filter(ee.Filter.eq('country_na', c)).geometry()
        for c in COUNTRIES_LIST
    }
    print('country_geoms rebuilt from LSIB')


def month_end_day(yr, mo):
    """Return the last day of a month as a string for GEE filterDate."""
    if mo == 2:
        return '28'
    return '30' if mo in [4, 6, 9, 11] else '31'


print('Extracting Sentinel-5P data — ~5-8 minutes...')

s5p_data = {prod: {c: {} for c in COUNTRIES_LIST} for prod in S5P_PRODUCTS}

for prod_name, meta in S5P_PRODUCTS.items():
    print(f'  {prod_name} ({meta["label"]})...')
    for country in COUNTRIES_LIST:
        geom = country_geoms[country]
        for yr in S5P_YEARS:
            for mo in range(1, 13):
                img = (
                    ee.ImageCollection(meta['collection'])
                    .filterDate(
                        f'{yr}-{mo:02d}-01',
                        f'{yr}-{mo:02d}-{month_end_day(yr, mo)}',
                    )
                    .select(meta['band'])
                    .mean()
                )
                result = img.reduceRegion(
                    reducer    = ee.Reducer.mean(),
                    geometry   = geom,
                    scale      = 7000,    # native TROPOMI resolution
                    maxPixels  = 1e10,
                    bestEffort = True,
                )
                try:
                    raw = result.getInfo().get(meta['band'])
                    s5p_data[prod_name][country][f'{yr}-{mo:02d}'] = (
                        round(float(raw) * meta['scale_factor'], 6)
                        if raw is not None else None
                    )
                except Exception:
                    s5p_data[prod_name][country][f'{yr}-{mo:02d}'] = None

        print(f'    {country}: done')

# Save CSVs
for prod_name in S5P_PRODUCTS:
    rows = [
        {'country': c, 'date': date_str, prod_name: val}
        for c in COUNTRIES_LIST
        for date_str, val in s5p_data[prod_name][c].items()
    ]
    out_path = f'data/processed/s5p_{prod_name.lower()}_country_2018_2024.csv'
    pd.DataFrame(rows).to_csv(out_path, index=False)
    print(f'Saved: {out_path}')

print('Sentinel-5P extraction complete')

### 8.2  Load and Prepare S5P Data Summaries

In [ ]:
# Peak fire season months (Jul-Oct) are the primary signal for biomass burning.
# We flag them here so every downstream aggregation can filter to peak-season easily.
COUNTRIES  = COUNTRIES_LIST
CLRS_C     = dict(zip(COUNTRIES, ['#e31a1c','#fd8d3c','#fecc5c','#41ab5d']))

PROD_META = {
    'CO'    : {'label': 'CO Column Density (mmol/m²)',   'color': '#d73027'},
    'NO2'   : {'label': 'NO₂ Column (µmol/m²)',          'color': '#fc8d59'},
    'AER_AI': {'label': 'Absorbing Aerosol Index (AAI)', 'color': '#fee090'},
}

s5p_dfs = {}
for prod in ['CO', 'NO2', 'AER_AI']:
    s5p_dfs[prod] = (
        pd.read_csv(f'data/processed/s5p_{prod.lower()}_country_2018_2024.csv',
                    parse_dates=['date'])
        .assign(
            year    = lambda d: d['date'].dt.year,
            month   = lambda d: d['date'].dt.month,
            is_peak = lambda d: d['month'].isin([7, 8, 9, 10]),
        )
    )

# Monthly climatology, peak-season means, annual means
s5p_clim   = {p: s5p_dfs[p].groupby(['country','month'])[p].mean().reset_index()
              for p in s5p_dfs}
s5p_peak   = {p: s5p_dfs[p][s5p_dfs[p]['is_peak']]
                  .groupby(['country','year'])[p].mean().reset_index()
              for p in s5p_dfs}
s5p_annual = {p: s5p_dfs[p].groupby(['country','year'])[p].mean().reset_index()
              for p in s5p_dfs}

print('S5P summaries ready')
for prod, agg in s5p_peak.items():
    print(f'  {prod}: {len(agg)} country-year records')


### Figures 5 & 6: S5P Monthly Climatology and Fire Correlation

In [ ]:
# ------ Figure 5: 3-row × 4-col climatology (product × country) ------------------
# The 3x4 grid is a deliberate choice: it lets you scan down a column to
# compare how the same country behaves across CO/NO2/AAI,
# or across a row to compare how the same gas responds in different countries.
fig5, axes5 = plt.subplots(
    3, 4, figsize=(18, 13),
    gridspec_kw={'hspace': 0.45, 'wspace': 0.35},
)
fig5.patch.set_facecolor('white')
fig5.suptitle(
    'Sentinel-5P Atmospheric Emissions — Monthly Climatology\n'
    'Country-wise Mean 2018–2024 | Fire Season (Jul–Oct) Highlighted',
    fontsize=14, fontweight='bold', y=1.01,
)

for ri, prod in enumerate(['CO', 'NO2', 'AER_AI']):
    clim = s5p_clim[prod]
    lbl  = PROD_META[prod]['label']

    for ci, country in enumerate(COUNTRIES):
        ax   = axes5[ri, ci]
        data = clim[clim['country'] == country].sort_values('month')
        months, vals = data['month'].values, data[prod].values

        ax.axvspan(6.5, 10.5, alpha=0.12, color='#e31a1c', zorder=0)

        bar_cols = ['#d73027' if m in [7,8,9,10]
                    else '#fc8d59' if m in [6,11]
                    else '#d9d9d9' for m in months]
        ax.bar(months, vals, color=bar_cols,
               edgecolor='white', linewidth=0.5, width=0.75)
        ax.plot(months, vals, 'o-', color='#252525',
                linewidth=1.5, markersize=3, zorder=3)

        ax.set_xticks(range(1, 13))
        ax.set_xticklabels(MONTH_LABELS, fontsize=7, rotation=45)
        ax.tick_params(axis='y', labelsize=7)
        ax.set_title(country, fontsize=9, fontweight='bold', pad=3)
        ax.grid(axis='y', linestyle='--', alpha=0.4, linewidth=0.6)
        ax.spines[['top', 'right']].set_visible(False)
        if ci == 0:
            ax.set_ylabel(lbl, fontsize=8)

# Row labels (rotated product names on left margin)
for ri, (prod, meta) in enumerate(PROD_META.items()):
    fig5.text(0.005, 0.78 - ri * 0.30, prod,
              fontsize=12, fontweight='bold', va='center',
              color=meta['color'], rotation=90)

fig5.text(0.5, -0.01,
          'Data: Copernicus Sentinel-5P TROPOMI | ~7 km | GEE | '
          'Red bars = fire season months (Jul–Oct)',
          ha='center', fontsize=8, color='#555', style='italic')

plt.savefig('outputs/figures/fig5_s5p_monthly_climatology.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 5 saved')


In [ ]:
import matplotlib.gridspec as gridspec

# ------ Fire peak-season totals per country-year (for correlation) ----------------
# Build peak-season fire totals per country per year (for the correlation axis).
# Using only Jul-Oct burned area so it's comparable to S5P peak-season values.
fire_peak = {
    country: {
        yr: df[(df['year'] == yr) & df['month'].isin([7, 8, 9, 10])]
              ['burned_area_km2'].sum() / 1e3   # k km²
        for yr in S5P_YEARS
    }
    for country in COUNTRIES
}

# -----Figure 6: Peak-season trends + fire correlation ------------------------------
# Figure 6: time series + secondary fire axis + Pearson r.
# The strong negative CO-fire correlation in most countries is counter-intuitive
# at first glance -- it actually reflects the long CO lifetime; CO accumulates
# and is then transported downwind, so peak-season country averages don't
# necessarily track local fire at the same time.
fig6 = plt.figure(figsize=(18, 12))
fig6.patch.set_facecolor('white')
gs   = gridspec.GridSpec(3, 5, figure=fig6,
                          hspace=0.5, wspace=0.42,
                          width_ratios=[1, 1, 1, 1, 0.05])
fig6.suptitle(
    'Fire Season (Jul–Oct) Atmospheric Emissions — Trends & Fire Correlation\n'
    'Sentinel-5P TROPOMI × MODIS MCD64A1 | 2018–2024',
    fontsize=14, fontweight='bold', y=1.01,
)

for ri, prod in enumerate(['CO', 'NO2', 'AER_AI']):
    pk_df = s5p_peak[prod]
    col   = PROD_META[prod]['color']
    lbl   = PROD_META[prod]['label']

    for ci, country in enumerate(COUNTRIES):
        ax    = fig6.add_subplot(gs[ri, ci])
        cdata = pk_df[pk_df['country'] == country].sort_values('year')
        yrs   = cdata['year'].values
        vals  = cdata[prod].values

        # OLS trend
        if len(yrs) > 2:
            p_fit = np.poly1d(np.polyfit(yrs, vals, 1))
            ax.fill_between(yrs, vals, p_fit(yrs), alpha=0.10, color=col)
            ax.plot(yrs, p_fit(yrs), '--', color=col, linewidth=1.2, alpha=0.7)

        ax.plot(yrs, vals, 'o-', color=col, linewidth=2, markersize=5,
                markerfacecolor='white', markeredgewidth=1.5)

        # Secondary axis: fire area
        ax2 = ax.twinx()
        fv  = [fire_peak[country].get(yr, 0) for yr in yrs]
        ax2.fill_between(yrs, fv, alpha=0.15, color='#777')
        ax2.plot(yrs, fv, 's--', color='#555', linewidth=1, markersize=3, alpha=0.6)
        ax2.set_ylabel('Burned\n(k km²)', fontsize=7, color='#666', labelpad=2)
        ax2.tick_params(axis='y', labelsize=6, colors='#666')
        ax2.spines[['top']].set_visible(False)

        r = np.corrcoef(vals, fv)[0, 1] if len(vals) > 2 else float('nan')
        ax.set_title(f'{country}\nr={r:.2f}', fontsize=9, fontweight='bold', pad=3)
        ax.set_xticks(yrs)
        ax.set_xticklabels([str(y)[2:] for y in yrs], fontsize=7)
        ax.tick_params(axis='y', labelsize=7)
        ax.grid(axis='y', linestyle='--', alpha=0.35, linewidth=0.6)
        ax.spines[['top', 'right']].set_visible(False)
        if ci == 0:
            ax.set_ylabel(lbl, fontsize=8)

fig6.text(0.5, -0.01,
          'Lines = S5P emission metric (left axis) | '
          'Grey fill = fire season burned area (right axis, k km²) | '
          'r = Pearson correlation',
          ha='center', fontsize=8, color='#555', style='italic')

plt.savefig('outputs/figures/fig6_s5p_fire_correlation.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 6 saved')


## Section 9: Forest Cover Loss Analysis (Hansen GFC)

### 9.1.  GEE Extraction: Annual Forest Loss per Country (2015-2023)

Hansen et al. (2013, *Science*) GFC product provides 30 m global tree cover and
annual loss year band.  We use the >=25% canopy cover threshold as the FAO-standard
definition of forest.  `lossyear` stores values 1-23 (= 2001-2023), so we subtract
2000 to get the actual year.

Note: GFC 2024 data wasn't available at extraction time -- analysis covers 2015-2023.
Scale is set to 100 m (upsampled from native 30 m) to keep GEE compute manageable
without meaningfully changing area statistics.

In [ ]:
gfc         = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')
forest_mask = gfc.select('treecover2000').gte(25)   # FAO ≥25% threshold
loss_yr     = gfc.select('lossyear')                # 1 = 2001, 23 = 2023

GFC_YEARS = list(range(2015, 2024))   # 2015–2023 (year codes 15–23)

forest_loss_data = {c: {} for c in COUNTRIES_LIST}
forest_area_data = {c: 0   for c in COUNTRIES_LIST}

print('Extracting forest cover loss — ~4-6 minutes...')

for country in COUNTRIES_LIST:
    geom = country_geoms[country]

    # Total forested area in year 2000
    tot_result = (
        forest_mask
        .multiply(ee.Image.pixelArea().divide(1e6))
        .reduceRegion(
            reducer    = ee.Reducer.sum(),
            geometry   = geom,
            scale      = 100,
            maxPixels  = 1e12,
            bestEffort = True,
        )
    )
    forest_area_data[country] = round(
        float(tot_result.getInfo().get('treecover2000', 0)), 1)

    # Annual loss by year code
    for yr in GFC_YEARS:
        yr_mask  = loss_yr.eq(yr - 2000).And(forest_mask)
        area_img = yr_mask.multiply(ee.Image.pixelArea().divide(1e6))
        result   = area_img.reduceRegion(
            reducer    = ee.Reducer.sum(),
            geometry   = geom,
            scale      = 100,
            maxPixels  = 1e12,
            bestEffort = True,
        )
        forest_loss_data[country][yr] = round(
            float(result.getInfo().get('lossyear', 0)), 2)

    print(f'  {country}: 2000 forest = {forest_area_data[country]:,.0f} km²')

# Assemble and save DataFrame
df_forest = (
    pd.DataFrame([
        {'country': c, 'year': yr, 'forest_loss_km2': v,
         'forest_area_2000_km2': forest_area_data[c]}
        for c, yd in forest_loss_data.items()
        for yr, v in yd.items()
    ])
    .assign(loss_pct = lambda d:
            (d['forest_loss_km2'] / d['forest_area_2000_km2'] * 100).round(4))
)
df_forest.to_csv('data/processed/forest_cover_loss_country_2015_2023.csv', index=False)
print('Forest loss data saved')


### Figures 7 & 8: Forest Loss Trends and Fire-Forest Nexus

In [ ]:
df_forest = pd.read_csv(
    'data/processed/forest_cover_loss_country_2015_2023.csv',
    dtype={'year': int},
)

# ----- Figure 7: Four-panel country-wise forest loss ---------------------------
# Figure 7: four-panel country-wise forest loss.
# Dual y-axis: absolute loss (bars) on left + % of 2000 forest area (dots) on right.
# The % axis matters because Malawi has much less forest than Mozambique,
# so absolute numbers are misleading on their own.
fig7, axes7 = plt.subplots(
    2, 2, figsize=(14, 10),
    gridspec_kw={'hspace': 0.45, 'wspace': 0.35},
)
fig7.patch.set_facecolor('white')
fig7.suptitle(
    'Annual Forest Cover Loss — Country-wise Analysis (2015–2023)\n'
    'Hansen GFC v1.11 | 30m | Tree cover ≥25% threshold',
    fontsize=13, fontweight='bold',
)

for i, country in enumerate(COUNTRIES_LIST):
    ax  = axes7.flatten()[i]
    cdf = df_forest[df_forest['country'] == country].sort_values('year')
    yrs = cdf['year'].values
    lv  = cdf['forest_loss_km2'].values
    lp  = cdf['loss_pct'].values
    col = list(CLRS_C.values())[i]

    ax.bar(yrs, lv, color=col, alpha=0.8, edgecolor='white', linewidth=0.6)
    ax.plot(yrs, lv, 'o-', color='#333', linewidth=1.5, markersize=4, zorder=3)
    p_fit = np.poly1d(np.polyfit(yrs, lv, 1))
    ax.plot(yrs, p_fit(yrs), '--', color='#555', linewidth=1.2, alpha=0.7,
            label='OLS trend')

    for yr, v in zip(yrs, lv):
        ax.text(yr, v + lv.max() * 0.02, f'{v:.0f}',
                ha='center', va='bottom', fontsize=7, color='#333')

    ax2 = ax.twinx()
    ax2.plot(yrs, lp, 's:', color='#666', linewidth=1, markersize=3, alpha=0.6)
    ax2.set_ylabel('Loss %\n(of 2000 forest)', fontsize=7,
                   color='#666', labelpad=2)
    ax2.tick_params(axis='y', labelsize=6, colors='#666')
    ax2.yaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, _: f'{x:.3f}%'))
    ax2.spines[['top']].set_visible(False)

    fa  = cdf['forest_area_2000_km2'].iloc[0]
    ax.set_title(
        f'{country}\nForest 2000: {fa:,.0f} km²  |  Total loss: {lv.sum():,.0f} km²',
        fontsize=9, fontweight='bold', pad=4,
    )
    ax.set_xticks(yrs)
    ax.set_xticklabels([str(y) for y in yrs], rotation=45, fontsize=7)
    ax.set_ylabel('Forest Loss (km²)', fontsize=8)
    ax.tick_params(axis='y', labelsize=7)
    ax.grid(axis='y', linestyle='--', alpha=0.4, linewidth=0.6)
    ax.spines[['top', 'right']].set_visible(False)

fig7.text(0.5, -0.01,
          'Data: Hansen GFC v1.11 | UMD/GEE | '
          'Dashed = OLS trend | Grey = % of 2000 forest area',
          ha='center', fontsize=8, color='#555', style='italic')

plt.savefig('outputs/figures/fig7_forest_loss_country.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 7 saved')


In [ ]:
# Figure 8: fire-forest nexus.
# The scatter (bottom panels) is the key result: negative r values mean
# years with more fire are years with more forest loss -- consistent with
# the agricultural expansion hypothesis where smallholder burning opens
# land for cultivation, rather than the post-fire recovery hypothesis.
# Malawi shows the strongest coupling (r ~ -0.88) which makes sense given
# its high population density and very limited remaining forest.
GFC_OVERLAP = list(range(2015, 2024))    # years where both datasets overlap

# ----- Figure 8: Fire × Forest Loss correlation ------------------------------
fig8, axes8 = plt.subplots(
    2, 4, figsize=(18, 9),
    gridspec_kw={'hspace': 0.45, 'wspace': 0.38},
)
fig8.patch.set_facecolor('white')
fig8.suptitle(
    'Fire–Forest Loss Nexus: Burned Area vs Annual Forest Loss\n'
    'MODIS MCD64A1 × Hansen GFC | 2015–2023 | Country-wise',
    fontsize=13, fontweight='bold',
)

for ci, country in enumerate(COUNTRIES_LIST):
    col  = list(CLRS_C.values())[ci]
    cdf  = df_forest[df_forest['country'] == country].sort_values('year')

    fire = np.array([df[df['year'] == yr]['burned_area_km2'].sum() / 1e3
                     for yr in GFC_OVERLAP])
    loss = np.array([cdf[cdf['year'] == yr]['forest_loss_km2'].values[0]
                     for yr in GFC_OVERLAP
                     if len(cdf[cdf['year'] == yr]) > 0])
    yrs  = np.array(GFC_OVERLAP[:len(loss)])

    # ---- Top row: time series overlay --------------------------------------
    ax_t  = axes8[0, ci]
    ax_t2 = ax_t.twinx()

    ax_t.fill_between(yrs, fire, alpha=0.25, color=col)
    ax_t.plot(yrs, fire, 'o-', color=col, linewidth=2, markersize=5)
    ax_t2.fill_between(yrs, loss, alpha=0.20, color='#2ca25f')
    ax_t2.plot(yrs, loss, 's--', color='#006d2c', linewidth=1.5, markersize=4)

    ax_t.set_title(country, fontsize=10, fontweight='bold', pad=3)
    ax_t.set_ylabel('Burned (k km²)',  fontsize=8, color=col)
    ax_t2.set_ylabel('Forest Loss (km²)', fontsize=8, color='#006d2c', labelpad=2)
    ax_t.set_xticks(yrs)
    ax_t.set_xticklabels([str(y)[2:] for y in yrs], fontsize=7)
    ax_t.tick_params(axis='y', labelsize=7, colors=col)
    ax_t2.tick_params(axis='y', labelsize=7, colors='#006d2c')
    ax_t.grid(axis='y', linestyle='--', alpha=0.3)
    ax_t.spines[['top']].set_visible(False)
    ax_t2.spines[['top']].set_visible(False)

    # ── Bottom row: scatter + OLS regression ──────────────────────────────
    ax_s = axes8[1, ci]
    ax_s.scatter(fire, loss, color=col, s=55,
                 edgecolors='#333', linewidths=0.8, zorder=3)

    for yr, fx, lx in zip(yrs, fire, loss):
        ax_s.annotate(str(yr)[2:], (fx, lx), fontsize=7, color='#333',
                      xytext=(3, 3), textcoords='offset points')

    if len(fire) > 2:
        p_fit = np.poly1d(np.polyfit(fire, loss, 1))
        xr    = np.linspace(fire.min(), fire.max(), 50)
        ax_s.plot(xr, p_fit(xr), '--', color='#555', linewidth=1.3, alpha=0.8)
        r = np.corrcoef(fire, loss)[0, 1]
        r_color = '#e31a1c' if r > 0 else '#2166ac'
        ax_s.text(0.05, 0.93, f'r = {r:.2f}', transform=ax_s.transAxes,
                  fontsize=10, fontweight='bold',
                  color='#333' if abs(r) < 0.5 else r_color)

    ax_s.set_xlabel('Annual Burned Area (k km²)', fontsize=8)
    ax_s.set_ylabel('Forest Loss (km²)',           fontsize=8)
    ax_s.tick_params(labelsize=7)
    ax_s.grid(linestyle='--', alpha=0.35)
    ax_s.spines[['top', 'right']].set_visible(False)

fig8.text(0.5, -0.01,
          'Top: temporal overlap of burned area & forest loss | '
          'Bottom: scatter regression (r = Pearson) | '
          'Data: MODIS MCD64A1 + Hansen GFC v1.11',
          ha='center', fontsize=8, color='#555', style='italic')

plt.savefig('outputs/figures/fig8_fire_forest_correlation.png',
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print('Figure 8 saved')


## Section 10: Interactive Folium Map

> **Run this section after Sections 7, 8 and 9** so all three data tabs (By Country,
> Emissions, Forest) are populated.  If you run it earlier the tabs will show a
> placeholder message -- just re-run this cell once the data is ready.

The map uses pure-Python SVG generators for all charts (no external JS CDNs) so
it works completely offline once saved.  The HTML file is ~12-15 MB.

In [ ]:
import folium
from folium.plugins import HeatMap, MiniMap, Fullscreen, MousePosition, MeasureControl
from IPython.display import HTML
import geopandas as gpd
import numpy as np
import rasterio
import json as _json

# ═══════════════════════════════════════════════════════════════════════
# STEP 1 — Load raster & build heat data
# ═══════════════════════════════════════════════════════════════════════
with rasterio.open('data/raw/fire_frequency_2015_2024.tif') as src:
    fire_arr  = src.read(1).astype(float)
    transform = src.transform

rows_r, cols_r = np.where(fire_arr > 1)
weights_r      = fire_arr[rows_r, cols_r]
lons_r, lats_r = rasterio.transform.xy(transform, rows_r, cols_r)

MAX_POINTS = 60000
if len(lons_r) > MAX_POINTS:
    idx       = np.random.choice(len(lons_r), MAX_POINTS, replace=False)
    lons_r    = np.array(lons_r)[idx]
    lats_r    = np.array(lats_r)[idx]
    weights_r = weights_r[idx]

w_norm    = (weights_r - weights_r.min()) / (weights_r.max() - weights_r.min() + 1e-9)
heat_data = [[float(la), float(lo), float(w)]
             for la, lo, w in zip(lats_r, lons_r, w_norm)]

_ann = df.groupby('year')['burned_area_km2'].sum()
yearly_heat = {}
for yr in range(2015, 2025):
    scale = _ann.get(yr, 0) / _ann.max()
    yearly_heat[yr] = [[la, lo, float(w * scale)]
                       for la, lo, w in zip(lats_r, lons_r, w_norm)]

# ═══════════════════════════════════════════════════════════════════════
# STEP 2 — Admin boundaries
# ═══════════════════════════════════════════════════════════════════════
world  = gpd.read_file(
    'https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_admin_0_countries.zip')
study  = world[world['NAME'].isin(['Zambia','Tanzania','Malawi','Mozambique'])]
others = world[world['NAME'].isin([
    'Zimbabwe','Angola','Kenya','Uganda','Rwanda','Burundi',
    'Democratic Republic of the Congo','Namibia','Botswana','South Africa'
])]

# ═══════════════════════════════════════════════════════════════════════
# STEP 3 — Folium map & layers
# ═══════════════════════════════════════════════════════════════════════
sat_url = ('https://server.arcgisonline.com/ArcGIS/rest/services/'
           'World_Imagery/MapServer/tile/{z}/{y}/{x}')

m = folium.Map(location=[-12, 32], zoom_start=5, tiles=None, prefer_canvas=True)
folium.TileLayer(tiles=sat_url, attr='Esri', name='Satellite Imagery').add_to(m)
folium.TileLayer(
    tiles='https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}{r}.png',
    attr='CartoDB', name='CartoDB Light').add_to(m)
folium.TileLayer(
    tiles='https://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}{r}.png',
    attr='CartoDB', name='Dark Mode').add_to(m)

folium.GeoJson(
    study.__geo_interface__, name='Study Area Boundary (4 Countries)',
    style_function=lambda x: {'fillColor':'none','color':'#ffdd00','weight':2.5},
    tooltip=folium.GeoJsonTooltip(fields=['NAME'], aliases=['Country:'])
).add_to(m)
folium.GeoJson(
    others.__geo_interface__, name='Neighboring Countries',
    style_function=lambda x: {'fillColor':'none','color':'#ffffff',
                               'weight':0.8,'dashArray':'4 4'},
    tooltip=folium.GeoJsonTooltip(fields=['NAME'], aliases=['Country:'])
).add_to(m)

grad = {0.0:'#ffffb2',0.25:'#fecc5c',0.5:'#fd8d3c',0.75:'#e31a1c',1.0:'#800026'}
HeatMap(heat_data, name='All Years (2015-2024)',
        min_opacity=0.35, radius=7, blur=5, max_zoom=12, gradient=grad).add_to(m)
for yr in range(2015, 2025):
    HeatMap(yearly_heat[yr], name=f'Year: {yr}',
            min_opacity=0.35, radius=7, blur=5, max_zoom=12,
            gradient=grad, show=False).add_to(m)

Fullscreen(position='topleft').add_to(m)
MiniMap(toggle_display=True, position='bottomright', zoom_level_offset=-6).add_to(m)
MousePosition(position='bottomleft', separator=' | Lon: ', prefix='Lat: ', num_digits=4).add_to(m)
MeasureControl(position='topleft', primary_length_unit='kilometers').add_to(m)
folium.LayerControl(collapsed=True, position='topright').add_to(m)

# ═══════════════════════════════════════════════════════════════════════
# STEP 4 — Prepare fire data
# ═══════════════════════════════════════════════════════════════════════
countries    = ['Zambia','Tanzania','Malawi','Mozambique']
country_clrs = ['#e31a1c','#fd8d3c','#fecc5c','#41ab5d']
years_list   = list(range(2015, 2025))

cy        = {c:[round(country_yearly[c].get(yr,0),0) for yr in years_list] for c in countries}
c_totals  = {c:sum(cy[c]) for c in countries}
c_means   = {c:round(sum(cy[c])/10,0) for c in countries}
c_peak_yr = {c:years_list[cy[c].index(max(cy[c]))] for c in countries}
c_peak_v  = {c:max(cy[c]) for c in countries}

ann_totals  = [round(_ann.get(yr,0)/1e3,1) for yr in years_list]
mean_ann    = round(_ann.mean()/1e3,1)
peak_mo     = monthly_clim.loc[monthly_clim['mean'].idxmax(),'month_label']
peak_yr_v   = int(_ann.idxmax())
monthly_v   = [round(v/1e3,1) for v in monthly_clim['mean'].values]
grand_total = int(sum(c_totals.values())/1e3)

# ═══════════════════════════════════════════════════════════════════════
# STEP 5 — Prepare S5P data (requires Section 8 to have been run)
# ═══════════════════════════════════════════════════════════════════════
S5P_AVAILABLE = 's5p_dfs' in dir()

if S5P_AVAILABLE:
    s5p_prod_meta = {
        'CO':     {'label':'CO Column (mmol/m\u00b2)', 'color':'#d73027', 'unit':'mmol/m\u00b2'},
        'NO2':    {'label':'NO2 Column (\u00b5mol/m\u00b2)','color':'#fc8d59','unit':'\u00b5mol/m\u00b2'},
        'AER_AI': {'label':'Aerosol Index (AAI)',       'color':'#fee090', 'unit':'AAI'},
    }
    # Peak-season (Jul-Oct) mean per country per year
    s5p_peak_vals = {}
    s5p_years_list = list(range(2018, 2025))
    for prod in ['CO','NO2','AER_AI']:
        df_p = s5p_dfs[prod]
        pk   = df_p[df_p['is_peak']].groupby(['country','year'])[prod].mean()
        s5p_peak_vals[prod] = {
            c: [round(pk.get((c, yr), float('nan')), 5) for yr in s5p_years_list]
            for c in countries
        }
    # Annual mean per country per year
    s5p_annual_vals = {}
    for prod in ['CO','NO2','AER_AI']:
        df_p = s5p_dfs[prod]
        ann  = df_p.groupby(['country','year'])[prod].mean()
        s5p_annual_vals[prod] = {
            c: [round(ann.get((c, yr), float('nan')), 5) for yr in s5p_years_list]
            for c in countries
        }
    # Overall peak-season country mean (single number for KPI)
    s5p_country_peak_mean = {}
    for prod in ['CO','NO2','AER_AI']:
        s5p_country_peak_mean[prod] = {
            c: round(float(sum(v for v in s5p_peak_vals[prod][c]
                              if not (isinstance(v, float) and v != v))
                          / max(1, sum(1 for v in s5p_peak_vals[prod][c]
                                      if not (isinstance(v, float) and v != v)))), 5)
            for c in countries
        }
    print("S5P data prepared for Folium map")
else:
    print("WARNING: S5P data not found — run Section 8 first, then re-run this cell")
    print("Emissions tab will show placeholder message")

# ═══════════════════════════════════════════════════════════════════════
# STEP 6 — Prepare Forest Loss data (requires Section 9 to have been run)
# ═══════════════════════════════════════════════════════════════════════
FOREST_AVAILABLE = 'df_forest' in dir()

if FOREST_AVAILABLE:
    gfc_years     = list(range(2015, 2024))
    forest_loss_v = {
        c: [round(df_forest.loc[
                (df_forest['country']==c) & (df_forest['year']==yr),
                'forest_loss_km2'].values[0], 1)
            if len(df_forest.loc[
                (df_forest['country']==c) & (df_forest['year']==yr)]) > 0
            else 0.0
            for yr in gfc_years]
        for c in countries
    }
    forest_area_2000 = {
        c: round(df_forest.loc[df_forest['country']==c,
                                'forest_area_2000_km2'].iloc[0], 0)
        for c in countries
    }
    forest_total_loss = {
        c: round(sum(forest_loss_v[c]), 1) for c in countries
    }
    forest_peak_yr = {
        c: gfc_years[forest_loss_v[c].index(max(forest_loss_v[c]))]
        for c in countries
    }
    print("Forest data prepared for Folium map")
else:
    print("WARNING: Forest data not found — run Section 9 first, then re-run this cell")
    print("Forest tab will show placeholder message")

# ═══════════════════════════════════════════════════════════════════════
# STEP 7 — SVG chart generators (pure Python, zero dependencies)
# ═══════════════════════════════════════════════════════════════════════

def svg_bar_chart(values, labels, mean_line, w=325, h=130, bar_color_fn=None):
    pad_l, pad_r, pad_t, pad_b = 42, 10, 12, 28
    cw = w - pad_l - pad_r
    ch = h - pad_t - pad_b
    n  = len(values)
    vmax = max(values) * 1.12 if max(values) > 0 else 1
    def ys(v): return pad_t + ch - (v / vmax) * ch
    bw = cw / n; gap = bw * 0.18; bwr = bw - gap * 2
    parts = [f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg" style="overflow:visible">']
    for i in range(5):
        v = vmax * i / 4; yp = ys(v)
        parts.append(f'<line x1="{pad_l}" y1="{yp:.1f}" x2="{pad_l+cw}" y2="{yp:.1f}" stroke="#1e1e1e" stroke-width="1"/>')
        parts.append(f'<text x="{pad_l-4}" y="{yp+4:.1f}" fill="#666" font-size="8" text-anchor="end">{v:.0f}k</text>')
    for i,(v,lbl) in enumerate(zip(values, labels)):
        x = pad_l + i*bw + gap; yv = ys(v); bh = ch-(yv-pad_t)
        col = bar_color_fn(v) if bar_color_fn else '#fd8d3c'
        parts.append(f'<rect x="{x:.1f}" y="{yv:.1f}" width="{bwr:.1f}" height="{bh:.1f}" fill="{col}" rx="2" opacity="0.88"/>')
        parts.append(f'<text x="{x+bwr/2:.1f}" y="{pad_t+ch+13}" fill="#777" font-size="8" text-anchor="middle">{str(lbl)[2:]}</text>')
        if bh > 16:
            parts.append(f'<text x="{x+bwr/2:.1f}" y="{yv-3:.1f}" fill="#aaa" font-size="7" text-anchor="middle">{v:.0f}k</text>')
    ym = ys(mean_line)
    parts.append(f'<line x1="{pad_l}" y1="{ym:.1f}" x2="{pad_l+cw}" y2="{ym:.1f}" stroke="#ffdd00" stroke-width="1.5" stroke-dasharray="5,3"/>')
    parts.append(f'<text x="{pad_l+cw-2}" y="{ym-4:.1f}" fill="#ffdd00" font-size="8" text-anchor="end">Mean {mean_line}k</text>')
    parts.append('</svg>')
    return ''.join(parts)


def svg_line_chart(values, labels, w=325, h=110, line_color='#e31a1c'):
    pad_l, pad_r, pad_t, pad_b = 42, 10, 12, 24
    cw = w - pad_l - pad_r; ch = h - pad_t - pad_b
    n  = len(values)
    vmax = max(values) * 1.15 if max(values) > 0 else 1
    pk   = max(values)
    def xs(i): return pad_l + i * cw / (n-1)
    def ys(v): return pad_t + ch - (v / vmax) * ch
    parts = [f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg" style="overflow:visible">']
    for i in range(5):
        v = vmax * i / 4; yp = ys(v)
        parts.append(f'<line x1="{pad_l}" y1="{yp:.1f}" x2="{pad_l+cw}" y2="{yp:.1f}" stroke="#1e1e1e" stroke-width="1"/>')
        parts.append(f'<text x="{pad_l-4}" y="{yp+4:.1f}" fill="#666" font-size="8" text-anchor="end">{v:.0f}k</text>')
    poly = (f'{pad_l},{ys(0):.1f} ' +
            ' '.join(f'{xs(i):.1f},{ys(v):.1f}' for i,v in enumerate(values)) +
            f' {xs(n-1):.1f},{ys(0):.1f}')
    parts.append(f'<polygon points="{poly}" fill="{line_color}" opacity="0.12"/>')
    lpts = ' '.join(f'{xs(i):.1f},{ys(v):.1f}' for i,v in enumerate(values))
    parts.append(f'<polyline points="{lpts}" fill="none" stroke="{line_color}" stroke-width="2.5" stroke-linejoin="round"/>')
    for i,(v,lbl) in enumerate(zip(values,labels)):
        x,y = xs(i),ys(v); ipk = (v==pk)
        r = 5 if ipk else 3; dc = '#ffdd00' if ipk else '#fc8d59'
        parts.append(f'<circle cx="{x:.1f}" cy="{y:.1f}" r="{r}" fill="{dc}"/>')
        if ipk:
            parts.append(f'<circle cx="{x:.1f}" cy="{y:.1f}" r="{r+2}" fill="none" stroke="#ffdd00" stroke-width="1.5"/>')
            parts.append(f'<text x="{x:.1f}" y="{y-9:.1f}" fill="#ffdd00" font-size="8" text-anchor="middle">Peak</text>')
        parts.append(f'<text x="{x:.1f}" y="{pad_t+ch+14}" fill="#777" font-size="8" text-anchor="middle">{lbl[:3]}</text>')
    parts.append('</svg>')
    return ''.join(parts)


def svg_grouped_bar(data_dict, labels, colors, w=325, h=195):
    pad_l, pad_r, pad_t, pad_b = 42, 10, 12, 52
    cw = w-pad_l-pad_r; ch = h-pad_t-pad_b
    keys = list(data_dict.keys()); ng = len(labels); ns = len(keys)
    all_v = [v for vals in data_dict.values() for v in vals]
    vmax  = max(all_v)*1.12 if max(all_v)>0 else 1
    def ys(v): return pad_t+ch-(v/vmax)*ch
    gw = cw/ng; bw = gw*0.85/ns; gap = gw*0.075
    parts = [f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg" style="overflow:visible">']
    for i in range(5):
        v=vmax*i/4; yp=ys(v)
        parts.append(f'<line x1="{pad_l}" y1="{yp:.1f}" x2="{pad_l+cw}" y2="{yp:.1f}" stroke="#1e1e1e" stroke-width="1"/>')
        parts.append(f'<text x="{pad_l-4}" y="{yp+4:.1f}" fill="#666" font-size="8" text-anchor="end">{v:.0f}k</text>')
    for gi,lbl in enumerate(labels):
        gx = pad_l+gi*gw+gap
        for si,(key,col) in enumerate(zip(keys,colors)):
            v=data_dict[key][gi]; x=gx+si*bw; yv=ys(v); bh=ch-(yv-pad_t)
            parts.append(f'<rect x="{x:.1f}" y="{yv:.1f}" width="{bw-1:.1f}" height="{bh:.1f}" fill="{col}" rx="1.5" opacity="0.85"/>')
        parts.append(f'<text x="{gx+(ns*bw)/2:.1f}" y="{pad_t+ch+13}" fill="#777" font-size="8" text-anchor="middle">{str(lbl)[2:]}</text>')
    lx = pad_l
    for si,(key,col) in enumerate(zip(keys,colors)):
        parts.append(f'<rect x="{lx:.1f}" y="{h-18}" width="9" height="9" fill="{col}" rx="1.5"/>')
        parts.append(f'<text x="{lx+12:.1f}" y="{h-10}" fill="#aaa" font-size="9">{key}</text>')
        lx += len(key)*6+22
    parts.append('</svg>')
    return ''.join(parts)


def svg_multiline(data_dict, labels, colors, w=325, h=150, y_fmt=None):
    """Multi-series line chart — for S5P trends."""
    pad_l, pad_r, pad_t, pad_b = 52, 10, 12, 42
    cw = w-pad_l-pad_r; ch = h-pad_t-pad_b
    keys = list(data_dict.keys())
    n    = len(labels)
    all_v = [v for vals in data_dict.values() for v in vals
             if v is not None and not (isinstance(v, float) and v != v)]
    if not all_v: return f'<svg width="{w}" height="{h}"><text x="10" y="20" fill="#666" font-size="10">No data</text></svg>'
    vmax = max(all_v)*1.15; vmin = min(all_v)*0.85 if min(all_v) > 0 else 0
    vrng = vmax-vmin if vmax-vmin > 0 else 1
    def xs(i): return pad_l + i*cw/(n-1) if n > 1 else pad_l+cw/2
    def ys(v): return pad_t + ch - (v-vmin)/vrng*ch
    parts = [f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg" style="overflow:visible">']
    for i in range(5):
        v = vmin+vrng*i/4; yp = ys(v)
        parts.append(f'<line x1="{pad_l}" y1="{yp:.1f}" x2="{pad_l+cw}" y2="{yp:.1f}" stroke="#1e1e1e" stroke-width="1"/>')
        lv = f'{v:.3f}' if y_fmt=='sci' else f'{v:.1f}'
        parts.append(f'<text x="{pad_l-3}" y="{yp+4:.1f}" fill="#666" font-size="7" text-anchor="end">{lv}</text>')
    for ki,(key,col) in enumerate(zip(keys,colors)):
        vals = data_dict[key]
        valid = [(i,v) for i,v in enumerate(vals)
                 if v is not None and not (isinstance(v,float) and v!=v)]
        if len(valid) < 2: continue
        pts_str = ' '.join(f'{xs(i):.1f},{ys(v):.1f}' for i,v in valid)
        parts.append(f'<polyline points="{pts_str}" fill="none" stroke="{col}" stroke-width="2" stroke-linejoin="round" opacity="0.9"/>')
        for i,v in valid:
            parts.append(f'<circle cx="{xs(i):.1f}" cy="{ys(v):.1f}" r="3" fill="{col}"/>')
    for i,lbl in enumerate(labels):
        parts.append(f'<text x="{xs(i):.1f}" y="{pad_t+ch+13}" fill="#777" font-size="8" text-anchor="middle">{str(lbl)[2:]}</text>')
    lx = pad_l
    for ki,(key,col) in enumerate(zip(keys,colors)):
        parts.append(f'<circle cx="{lx+4}" cy="{h-10}" r="4" fill="{col}"/>')
        parts.append(f'<text x="{lx+11}" y="{h-7}" fill="#aaa" font-size="9">{key}</text>')
        lx += len(key)*6+20
    parts.append('</svg>')
    return ''.join(parts)


def svg_hbar(values, labels, colors, w=325, h=130, unit=''):
    """Horizontal bar chart — for S5P country comparison."""
    pad_l, pad_r, pad_t, pad_b = 85, 50, 10, 10
    cw = w-pad_l-pad_r; ch = h-pad_t-pad_b
    n  = len(values)
    vmax = max(values)*1.15 if max(values)>0 else 1
    bh = ch/n*0.65; gap = ch/n*0.175
    parts = [f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg" style="overflow:visible">']
    for i in range(5):
        v=vmax*i/4; xp=pad_l+v/vmax*cw
        parts.append(f'<line x1="{xp:.1f}" y1="{pad_t}" x2="{xp:.1f}" y2="{pad_t+ch}" stroke="#1e1e1e" stroke-width="1"/>')
    for i,(v,lbl,col) in enumerate(zip(values,labels,colors)):
        y = pad_t + i*(ch/n) + gap
        bw_px = v/vmax*cw
        parts.append(f'<rect x="{pad_l}" y="{y:.1f}" width="{bw_px:.1f}" height="{bh:.1f}" fill="{col}" rx="2" opacity="0.85"/>')
        parts.append(f'<text x="{pad_l-5}" y="{y+bh/2+4:.1f}" fill="#ccc" font-size="9" text-anchor="end">{lbl}</text>')
        lbl_txt = f'{v:.4f}{unit}' if '.' in str(v) else f'{v:.1f}{unit}'
        parts.append(f'<text x="{pad_l+bw_px+4:.1f}" y="{y+bh/2+4:.1f}" fill="#aaa" font-size="8">{lbl_txt}</text>')
    parts.append('</svg>')
    return ''.join(parts)


# ═══════════════════════════════════════════════════════════════════════
# STEP 8 — Generate all SVG charts
# ═══════════════════════════════════════════════════════════════════════

# --- Fire charts ---
svg_annual = svg_bar_chart(
    ann_totals, years_list, mean_ann,
    bar_color_fn=lambda v: 'rgba(227,26,28,0.85)' if v > mean_ann
                           else 'rgba(253,141,60,0.75)'
)
svg_monthly = svg_line_chart(
    monthly_v,
    ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
)
cy_k = {c:[round(v/1e3,1) for v in cy[c]] for c in countries}
svg_country = svg_grouped_bar(cy_k, years_list, country_clrs, w=325, h=195)

# --- S5P charts ---
if S5P_AVAILABLE:
    # Multi-line: CO peak-season trend per country
    svg_co_trend = svg_multiline(
        {c: s5p_peak_vals['CO'][c] for c in countries},
        s5p_years_list, country_clrs, w=325, h=140, y_fmt='sci'
    )
    svg_no2_trend = svg_multiline(
        {c: s5p_peak_vals['NO2'][c] for c in countries},
        s5p_years_list, country_clrs, w=325, h=140, y_fmt='sci'
    )
    svg_aer_trend = svg_multiline(
        {c: s5p_peak_vals['AER_AI'][c] for c in countries},
        s5p_years_list, country_clrs, w=325, h=140, y_fmt='sci'
    )
    # Horizontal bars: country mean peak-season CO
    svg_co_hbar = svg_hbar(
        [s5p_country_peak_mean['CO'][c] for c in countries],
        countries, country_clrs, w=325, h=115, unit=''
    )
else:
    _ph = ('<div style="padding:20px;text-align:center;color:#555;">'
           'Run Section 8 (Sentinel-5P extraction)<br>'
           'then re-run this cell to load emissions data.</div>')
    svg_co_trend = svg_no2_trend = svg_aer_trend = svg_co_hbar = _ph

# --- Forest charts ---
if FOREST_AVAILABLE:
    # Grouped bar: annual forest loss per country
    forest_loss_k = {c:[round(v/1e3,3) for v in forest_loss_v[c]] for c in countries}
    svg_forest_grouped = svg_grouped_bar(
        forest_loss_k, gfc_years, country_clrs, w=325, h=175
    )
    # Line: cumulative loss per country
    forest_cum = {
        c: [round(sum(forest_loss_v[c][:i+1])/1e3, 2) for i in range(len(gfc_years))]
        for c in countries
    }
    svg_forest_cum = svg_multiline(
        forest_cum, gfc_years, country_clrs, w=325, h=130
    )
else:
    _ph2 = ('<div style="padding:20px;text-align:center;color:#555;">'
            'Run Section 9 (Hansen GFC extraction)<br>'
            'then re-run this cell to load forest data.</div>')
    svg_forest_grouped = svg_forest_cum = _ph2

# ═══════════════════════════════════════════════════════════════════════
# STEP 9 — Build HTML fragments
# ═══════════════════════════════════════════════════════════════════════

yr_hdr = ''.join([
    f"<th style='padding:3px 5px;color:#aaa;font-size:9px;"
    f"font-weight:normal;text-align:right;'>'{str(yr)[2:]}</th>"
    for yr in years_list
])
mat_rows = ''
for c in countries:
    cells = ''.join([
        f"<td style='padding:4px 5px;text-align:right;color:#ddd;font-size:9.5px;'>"
        f"{int(v/1e3)}k</td>" for v in cy[c]
    ])
    mat_rows += (
        f"<tr style='border-bottom:1px solid #1e1e1e;'>"
        f"<td style='padding:4px 8px;font-weight:600;color:#ffdd00;font-size:10px;"
        f"white-space:nowrap;'>{c}</td>{cells}"
        f"<td style='padding:4px 6px;text-align:right;color:#fe9929;"
        f"font-weight:700;font-size:10px;'>{int(c_totals[c]/1e3)}k</td></tr>"
    )
col_totals = ''.join([
    f"<td style='padding:4px 5px;text-align:right;font-size:9px;color:#888;"
    f"font-weight:600;'>{int(sum([cy[c][i] for c in countries])/1e3)}k</td>"
    for i in range(10)
])

c_cards = ''
for i,c in enumerate(countries):
    c_cards += (
        f"<div style='display:flex;align-items:center;justify-content:space-between;"
        f"background:#111;border-radius:6px;padding:7px 10px;margin-bottom:5px;"
        f"border-left:3px solid {country_clrs[i]};'>"
        f"<div><div style='font-size:11px;font-weight:600;color:#fff;'>{c}</div>"
        f"<div style='font-size:9px;color:#777;'>Peak: {c_peak_yr[c]} "
        f"&nbsp;({int(c_peak_v[c]/1e3)}k km\u00b2)</div></div>"
        f"<div style='text-align:right;'>"
        f"<div style='font-size:15px;font-weight:700;color:{country_clrs[i]};'>"
        f"{int(c_means[c]/1e3)}k</div>"
        f"<div style='font-size:8px;color:#666;'>km\u00b2 / yr avg</div>"
        f"</div></div>"
    )

trend_kpis = (
    f"<div style='background:#111;border-radius:6px;padding:8px 10px;"
    f"border-left:3px solid #e31a1c;'>"
    f"<div style='font-size:8px;color:#555;text-transform:uppercase;"
    f"letter-spacing:0.4px;'>10-yr Mean</div>"
    f"<div style='font-size:17px;font-weight:700;color:#fff;'>{mean_ann}k</div>"
    f"<div style='font-size:8px;color:#888;'>km\u00b2 / year</div></div>"
    f"<div style='background:#111;border-radius:6px;padding:8px 10px;"
    f"border-left:3px solid #fecc5c;'>"
    f"<div style='font-size:8px;color:#555;text-transform:uppercase;"
    f"letter-spacing:0.4px;'>Peak Month</div>"
    f"<div style='font-size:17px;font-weight:700;color:#fff;'>{peak_mo}</div>"
    f"<div style='font-size:8px;color:#888;'>dry season peak</div></div>"
    f"<div style='background:#111;border-radius:6px;padding:8px 10px;"
    f"border-left:3px solid #fd8d3c;'>"
    f"<div style='font-size:8px;color:#555;text-transform:uppercase;"
    f"letter-spacing:0.4px;'>Peak Year</div>"
    f"<div style='font-size:17px;font-weight:700;color:#fff;'>{peak_yr_v}</div>"
    f"<div style='font-size:8px;color:#888;'>highest burn</div></div>"
    f"<div style='background:#111;border-radius:6px;padding:8px 10px;"
    f"border-left:3px solid #41ab5d;'>"
    f"<div style='font-size:8px;color:#555;text-transform:uppercase;"
    f"letter-spacing:0.4px;'>Study Area</div>"
    f"<div style='font-size:13px;font-weight:700;color:#fff;'>4 Countries</div>"
    f"<div style='font-size:8px;color:#888;'>ZM \u2022 TZ \u2022 MW \u2022 MZ"
    f"</div></div>"
)

# S5P KPI cards
if S5P_AVAILABLE:
    s5p_kpis = ''
    for i,c in enumerate(countries):
        co_v  = s5p_country_peak_mean['CO'][c]
        no2_v = s5p_country_peak_mean['NO2'][c]
        aer_v = s5p_country_peak_mean['AER_AI'][c]
        s5p_kpis += (
            f"<div style='background:#111;border-radius:6px;padding:7px 10px;"
            f"margin-bottom:5px;border-left:3px solid {country_clrs[i]};'>"
            f"<div style='font-size:11px;font-weight:600;color:#fff;"
            f"margin-bottom:3px;'>{c}</div>"
            f"<div style='display:flex;gap:10px;font-size:9px;'>"
            f"<span style='color:#d73027;'>CO: {co_v:.4f} mmol/m\u00b2</span>"
            f"<span style='color:#fc8d59;'>NO\u2082: {no2_v:.4f} \u00b5mol/m\u00b2</span>"
            f"<span style='color:#fee090;'>AAI: {aer_v:.3f}</span>"
            f"</div></div>"
        )
else:
    s5p_kpis = ''

# Forest KPI cards
if FOREST_AVAILABLE:
    forest_cards = ''
    for i,c in enumerate(countries):
        fa  = forest_area_2000[c]
        tl  = forest_total_loss[c]
        pct = round(tl/fa*100, 3) if fa > 0 else 0
        pk  = forest_peak_yr[c]
        forest_cards += (
            f"<div style='display:flex;align-items:center;"
            f"justify-content:space-between;background:#111;border-radius:6px;"
            f"padding:7px 10px;margin-bottom:5px;"
            f"border-left:3px solid {country_clrs[i]};'>"
            f"<div><div style='font-size:11px;font-weight:600;color:#fff;'>{c}</div>"
            f"<div style='font-size:9px;color:#777;'>"
            f"Forest 2000: {fa:,.0f} km\u00b2 &nbsp;|&nbsp; Peak loss: {pk}</div>"
            f"</div>"
            f"<div style='text-align:right;'>"
            f"<div style='font-size:14px;font-weight:700;"
            f"color:{country_clrs[i]};'>{tl:.0f} km\u00b2</div>"
            f"<div style='font-size:8px;color:#666;'>{pct:.3f}% lost</div>"
            f"</div></div>"
        )
else:
    forest_cards = ''

# ═══════════════════════════════════════════════════════════════════════
# STEP 10 — Assemble HTML panels
# ═══════════════════════════════════════════════════════════════════════

css_block = """
<style>
  .fp  { position:fixed;z-index:1000;
         font-family:'Segoe UI',Arial,sans-serif; }
  .pc  { background:rgba(10,10,10,0.93);
         border:1px solid rgba(255,221,0,0.2);
         border-radius:10px;
         box-shadow:0 4px 18px rgba(0,0,0,0.65);color:#fff; }
  .tb  { background:none;border:none;color:#666;
         padding:6px 9px;cursor:pointer;
         font-size:10.5px;border-bottom:2px solid transparent;
         transition:color .15s; }
  .tb:hover { color:#ccc; }
  .tb.active { color:#ffdd00;border-bottom:2px solid #ffdd00; }
  .tc  { display:none; }
  .tc.active { display:block; }
  #apanel::-webkit-scrollbar { width:4px; }
  #apanel::-webkit-scrollbar-thumb { background:#2a2a2a;border-radius:2px; }
</style>
"""

title_p = """
<div class='fp pc' style='top:70px;left:55px;padding:13px 15px;width:230px;'>
  <div style='font-size:15px;font-weight:700;color:#e31a1c;margin-bottom:3px;'>
    10-Year Fire Frequency Map</div>
  <div style='font-size:11px;color:#ffdd00;font-weight:600;margin-bottom:2px;'>
    Zambia &bull; Tanzania &bull; Malawi &bull; Mozambique</div>
  <div style='font-size:9px;color:#555;'>
    MODIS MCD64A1 &nbsp;|&nbsp; 500m &nbsp;|&nbsp; 2015&ndash;2024</div>
  <hr style='border-color:#1e1e1e;margin:8px 0;'>
  <div style='font-size:9.5px;color:#aaa;line-height:1.9;'>
    <span style='color:#ffdd00;'>&#9654;</span>
    Toggle layers &mdash; top-right panel<br>
    <span style='color:#ffdd00;'>&#9654;</span>
    Zoom in to see local hotspots<br>
    <span style='color:#ffdd00;'>&#9654;</span>
    Hover borders for country names<br>
    <span style='color:#ffdd00;'>&#9654;</span>
    Ruler tool &mdash; measure distances
  </div>
</div>
"""

legend_p = """
<div class='fp pc' style='bottom:45px;left:15px;padding:12px 14px;width:220px;'>
  <div style='font-size:11px;font-weight:700;color:#ffdd00;margin-bottom:9px;'>
    Fire Frequency
    <span style='font-size:9px;font-weight:400;color:#555;'>
    &nbsp;(months burned / 10 yrs)</span>
  </div>
  <div style='background:linear-gradient(to right,
    #ffffb2,#fecc5c,#fd8d3c,#e31a1c,#800026);
    height:12px;border-radius:3px;margin-bottom:5px;'></div>
  <div style='display:flex;justify-content:space-between;
    font-size:8px;color:#666;margin-bottom:10px;'>
    <span>Low<br>1-2</span>
    <span style='text-align:center;'>Moderate<br>3-4</span>
    <span style='text-align:center;'>High<br>5-6</span>
    <span style='text-align:right;'>Extreme<br>9+</span>
  </div>
  <div style='font-size:9px;color:#555;border-top:1px solid #1e1e1e;
    padding-top:8px;line-height:2.1;'>
    <span style='display:inline-block;width:18px;height:2px;
      background:#ffdd00;vertical-align:middle;margin-right:6px;'></span>
    Study area boundary<br>
    <span style='display:inline-block;width:18px;height:0;
      border-top:1px dashed #888;vertical-align:middle;
      margin-right:6px;'></span>
    Neighboring countries
  </div>
  <div style='font-size:8px;color:#333;margin-top:7px;'>
    Source: NASA MODIS MCD64A1 v061
  </div>
</div>
"""

findings_tab = """
<div id='tf' class='tc'>
  <div style='font-size:10px;color:#666;margin-bottom:10px;'>
    Cross-validated with peer-reviewed literature</div>
  <div style='background:#111;border-radius:7px;padding:10px 12px;
    margin-bottom:8px;border-left:3px solid #e31a1c;'>
    <div style='font-size:10px;font-weight:700;color:#fff;margin-bottom:4px;'>
      &#10003;&nbsp; Peak fire season: July&ndash;October</div>
    <div style='font-size:9px;color:#aaa;line-height:1.6;'>
      Fires peak in early austral dry season across all 4 countries.
      Tanzania miombo woodlands peak in July; Zambia savanna peaks Aug&ndash;Sep.
      <br><span style='color:#444;font-size:8.5px;'>
      [NASA Earth Observatory; Carbon Balance &amp; Mgmt, 2015]</span></div>
  </div>
  <div style='background:#111;border-radius:7px;padding:10px 12px;
    margin-bottom:8px;border-left:3px solid #fd8d3c;'>
    <div style='font-size:10px;font-weight:700;color:#fff;margin-bottom:4px;'>
      &#10003;&nbsp; Zambia &amp; Mozambique: global fire hotspots</div>
    <div style='font-size:9px;color:#aaa;line-height:1.6;'>
      Zambia ranked 3rd globally &mdash; 86.6% of national area burned (2001&ndash;2020).
      Mozambique ranked 4th (73.5%). Combined 10-yr burned area &gt;1.2M km&sup2;.
      <br><span style='color:#444;font-size:8.5px;'>[PLoS ONE, 2025]</span></div>
  </div>
  <div style='background:#111;border-radius:7px;padding:10px 12px;
    margin-bottom:8px;border-left:3px solid #fecc5c;'>
    <div style='font-size:10px;font-weight:700;color:#fff;margin-bottom:4px;'>
      &#10003;&nbsp; Savanna &amp; miombo drive recurrence</div>
    <div style='font-size:9px;color:#aaa;line-height:1.6;'>
      Up to 50.6% of Tanzania miombo burns annually. Woody savanna shows
      highest fire recurrence across all 4 countries.
      <br><span style='color:#444;font-size:8.5px;'>
      [Carbon Balance &amp; Mgmt, 2015]</span></div>
  </div>
  <div style='background:#111;border-radius:7px;padding:10px 12px;
    margin-bottom:8px;border-left:3px solid #41ab5d;'>
    <div style='font-size:10px;font-weight:700;color:#fff;margin-bottom:4px;'>
      &#10003;&nbsp; Anthropogenic fire management</div>
    <div style='font-size:9px;color:#aaa;line-height:1.6;'>
      ~70% of Kafue NP burns annually via early dry-season management burns.
      Cropland mosaics in Malawi &amp; Mozambique show elevated recurrence.
      <br><span style='color:#444;font-size:8.5px;'>
      [NASA MODIS Gallery, 2023]</span></div>
  </div>
  <div style='background:#111;border-radius:7px;padding:10px 12px;
    border-left:3px solid #807dba;'>
    <div style='font-size:10px;font-weight:700;color:#fff;margin-bottom:4px;'>
      &#10003;&nbsp; Declining long-term trend</div>
    <div style='font-size:9px;color:#aaa;line-height:1.6;'>
      Sub-Saharan Africa: ~36,000 km&sup2;/yr decline (2001&ndash;2020),
      driven by agricultural expansion replacing traditional burning.
      <br><span style='color:#444;font-size:8.5px;'>[PLoS ONE, 2025]</span></div>
  </div>
</div>
"""

analytics_p = (
    "<div class='fp pc' style='top:300px;left:55px;padding:0;"
    "width:390px;max-height:calc(100vh - 380px);'>"

    # ── Tab bar — 6 tabs ──────────────────────────────────────────
    "<div style='display:flex;border-bottom:1px solid #1e1e1e;"
    "padding:4px 4px 0;background:rgba(16,16,16,0.98);"
    "border-radius:10px 10px 0 0;flex-wrap:nowrap;overflow-x:auto;'>"
    "<button class='tb active' onclick=\"swTab(this,'tt')\">Trend</button>"
    "<button class='tb' onclick=\"swTab(this,'tc2')\">By Country</button>"
    "<button class='tb' onclick=\"swTab(this,'tm')\">Matrix</button>"
    "<button class='tb' onclick=\"swTab(this,'te')\">Emissions</button>"
    "<button class='tb' onclick=\"swTab(this,'tfo')\">Forest</button>"
    "<button class='tb' onclick=\"swTab(this,'tf')\">Findings</button>"
    "<button onclick=\"document.getElementById('apanel').style.display='none'\""
    " style='margin-left:auto;background:none;border:none;color:#444;"
    "cursor:pointer;font-size:15px;padding:0 8px;line-height:1;flex-shrink:0;'>"
    "&times;</button></div>"

    # ── Panel body ────────────────────────────────────────────────
    "<div id='apanel' style='padding:13px;overflow-y:auto;"
    "max-height:calc(100vh - 445px);'>"

    # ── TAB: TREND ────────────────────────────────────────────────
    "<div id='tt' class='tc active'>"
    "<div style='font-size:10px;color:#888;margin-bottom:7px;'>"
    "Combined annual burned area &mdash; all 4 countries</div>"
    + svg_annual +
    "<hr style='border-color:#1e1e1e;margin:9px 0;'>"
    "<div style='font-size:10px;color:#888;margin-bottom:6px;'>"
    "Monthly fire climatology (10-year mean 2015&ndash;2024)</div>"
    + svg_monthly +
    "<hr style='border-color:#1e1e1e;margin:9px 0;'>"
    "<div style='display:grid;grid-template-columns:1fr 1fr;gap:7px;'>"
    + trend_kpis + "</div></div>"

    # ── TAB: BY COUNTRY ───────────────────────────────────────────
    "<div id='tc2' class='tc'>"
    "<div style='font-size:10px;color:#888;margin-bottom:7px;'>"
    "Annual burned area per country (000 km&sup2;)</div>"
    + svg_country +
    "<hr style='border-color:#1e1e1e;margin:9px 0;'>"
    + c_cards + "</div>"

    # ── TAB: MATRIX ───────────────────────────────────────────────
    "<div id='tm' class='tc'>"
    "<div style='font-size:10px;color:#888;margin-bottom:7px;'>"
    "Burned area: country &times; year &nbsp;(000 km&sup2;)</div>"
    "<div style='overflow-x:auto;'>"
    "<table style='border-collapse:collapse;width:100%;background:#0d0d0d;"
    "border-radius:6px;overflow:hidden;'>"
    "<thead><tr style='background:#161616;border-bottom:1px solid #2a2a2a;'>"
    "<th style='padding:6px 9px;text-align:left;color:#ffdd00;font-size:10px;'>"
    "Country</th>" + yr_hdr +
    "<th style='padding:4px 6px;color:#fe9929;font-size:9px;text-align:right;'>"
    "Total</th></tr></thead>"
    "<tbody>" + mat_rows + "</tbody>"
    "<tfoot><tr style='background:#161616;border-top:2px solid #2a2a2a;'>"
    "<td style='padding:5px 9px;font-size:9px;color:#888;font-weight:700;'>"
    "All 4</td>" + col_totals +
    f"<td style='padding:4px 6px;text-align:right;font-size:10px;"
    f"font-weight:700;color:#ffdd00;'>{grand_total}k</td>"
    "</tr></tfoot></table></div>"
    "<div style='font-size:8px;color:#333;margin-top:7px;'>"
    "Values in 000 km&sup2; &nbsp;|&nbsp; Source: NASA MODIS MCD64A1 v061"
    "</div></div>"

    # ── TAB: EMISSIONS (S5P) ──────────────────────────────────────
    "<div id='te' class='tc'>"
    "<div style='font-size:10px;color:#888;margin-bottom:4px;'>"
    "Peak fire season (Jul&ndash;Oct) CO column density per country</div>"
    "<div style='font-size:8px;color:#555;margin-bottom:6px;'>"
    "Sentinel-5P TROPOMI &nbsp;|&nbsp; ~7km &nbsp;|&nbsp; 2018&ndash;2024</div>"
    + svg_co_trend +
    "<hr style='border-color:#1e1e1e;margin:8px 0;'>"
    "<div style='font-size:10px;color:#888;margin-bottom:4px;'>"
    "NO\u2082 tropospheric column (fire season)</div>"
    + svg_no2_trend +
    "<hr style='border-color:#1e1e1e;margin:8px 0;'>"
    "<div style='font-size:10px;color:#888;margin-bottom:4px;'>"
    "Absorbing Aerosol Index &mdash; smoke plume intensity</div>"
    + svg_aer_trend +
    "<hr style='border-color:#1e1e1e;margin:8px 0;'>"
    "<div style='font-size:10px;color:#888;margin-bottom:6px;'>"
    "Fire-season mean emissions summary per country</div>"
    + s5p_kpis +
    "<div style='font-size:8px;color:#333;margin-top:8px;'>"
    "Data: Copernicus S5P TROPOMI OFFL | CO, NO\u2082, AAI | GEE</div>"
    "</div>"

    # ── TAB: FOREST ───────────────────────────────────────────────
    "<div id='tfo' class='tc'>"
    "<div style='font-size:10px;color:#888;margin-bottom:4px;'>"
    "Annual forest cover loss per country (km\u00b2)</div>"
    "<div style='font-size:8px;color:#555;margin-bottom:6px;'>"
    "Hansen GFC v1.11 &nbsp;|&nbsp; 30m &nbsp;|&nbsp; "
    "Tree cover \u226525% (2000 baseline)</div>"
    + svg_forest_grouped +
    "<hr style='border-color:#1e1e1e;margin:8px 0;'>"
    "<div style='font-size:10px;color:#888;margin-bottom:4px;'>"
    "Cumulative forest loss 2015&ndash;2023 (000 km\u00b2)</div>"
    + svg_forest_cum +
    "<hr style='border-color:#1e1e1e;margin:8px 0;'>"
    "<div style='font-size:10px;color:#888;margin-bottom:6px;'>"
    "Country summary &mdash; total loss 2015&ndash;2023</div>"
    + forest_cards +
    "<div style='font-size:8px;color:#333;margin-top:8px;'>"
    "Data: Hansen GFC v1.11 | UMD via GEE | 30m resolution</div>"
    "</div>"

    # ── TAB: FINDINGS ─────────────────────────────────────────────
    + findings_tab +

    "</div></div>"  # end apanel + analytics_p
)

attrib_p = """
<div class='fp' style='
  bottom:12px;left:50%;transform:translateX(-50%);
  background:rgba(0,0,0,0.50);
  backdrop-filter:blur(8px);-webkit-backdrop-filter:blur(8px);
  padding:7px 22px;border-radius:22px;
  box-shadow:0 2px 12px rgba(0,0,0,0.4);
  font-size:11px;white-space:nowrap;
  border:1px solid rgba(255,255,255,0.1);'>
  Prepared by:&nbsp;<b style='color:#fff;'>Ujjwal Kumar Swain</b>
  &nbsp;&bull;&nbsp;
  <a href='mailto:ujjwalks.iirs@gmail.com'
     style='color:#fecc5c;text-decoration:none;'>ujjwalks.iirs@gmail.com</a>
  &nbsp;&bull;&nbsp;
  <span style='color:#ccc;'>+91&nbsp;7978641480</span>
  &nbsp;&bull;&nbsp;
  <span style='color:#555;'>NASA MODIS + Copernicus S5P + Hansen GFC via GEE</span>
</div>
"""

tab_js = """
<script>
function swTab(btn, id) {
  document.querySelectorAll('.tc').forEach(t => t.classList.remove('active'));
  document.querySelectorAll('.tb').forEach(b => b.classList.remove('active'));
  document.getElementById(id).classList.add('active');
  btn.classList.add('active');
}
</script>
"""

# ═══════════════════════════════════════════════════════════════════════
# STEP 11 — Inject into map & render
# ═══════════════════════════════════════════════════════════════════════
full_html = css_block + title_p + legend_p + analytics_p + attrib_p + tab_js
m.get_root().html.add_child(folium.Element(full_html))

m.save('outputs/maps/interactive_fire_heatmap.html')
print("Interactive map saved: outputs/maps/interactive_fire_heatmap.html")
print(f"Tabs: Trend | By Country | Matrix | Emissions({'YES' if S5P_AVAILABLE else 'NO - run Sec 8'}) | Forest({'YES' if FOREST_AVAILABLE else 'NO - run Sec 9'}) | Findings")

with open('outputs/maps/interactive_fire_heatmap.html', 'r') as f:
    html_content = f.read()

HTML(html_content)

## Section 11: Summary Report

In [ ]:
sep = '=' * 72
print(sep)
print('  WILDFIRE, ATMOSPHERE & FOREST NEXUS — SUMMARY')
print('  Zambia · Tanzania · Malawi · Mozambique | 2015–2024')
print(sep)

print('\n  FIRE (MODIS MCD64A1 | 2015–2024)')
print(f'  10-year total burned area : {df["burned_area_km2"].sum():>14,.0f} km²')
print(f'  Mean annual burned area   : {annual_totals.mean():>14,.0f} km²/yr')
print(f'  Peak year                 : {annual_totals.idxmax():>14}')
print(f'  Peak month                : {monthly_clim.loc[monthly_clim["mean"].idxmax(),"month_label"]:>14}')

print('\n  ATMOSPHERE (Sentinel-5P TROPOMI | 2018–2024 | peak season Jul–Oct)')
for prod in ['CO', 'NO2', 'AER_AI']:
    df_p  = s5p_dfs[prod]
    pval  = df_p[df_p['is_peak']].groupby('country')[prod].mean()
    units = {'CO': 'mmol/m²', 'NO2': 'µmol/m²', 'AER_AI': 'AAI'}[prod]
    print(f'  {prod:8} peak-season country means ({units}):')
    for c in COUNTRIES_LIST:
        print(f'    {c:<12}: {pval.get(c, float("nan")):>8.4f}')

print('\n  FOREST (Hansen GFC v1.11 | 2015–2023)')
for c in COUNTRIES_LIST:
    cdf = df_forest[df_forest['country'] == c]
    fa  = cdf['forest_area_2000_km2'].iloc[0]
    tl  = cdf['forest_loss_km2'].sum()
    print(f'  {c:<12}: {fa:>10,.0f} km² (2000) | '
          f'{tl:>8,.1f} km² lost | {tl/fa*100:.2f}%')

print(f'\n  COMBINED FOREST LOSS: {df_forest["forest_loss_km2"].sum():,.0f} km² (2015–2023)')
print(sep)


## Section 12: Export All Outputs

In [ ]:
import zipfile

# Package everything into a single ZIP for download.
# Raw TIFs are excluded (too large) -- the recipient can regenerate them from
# the GEE tasks described in Section 3.2.
ZIP_NAME = 'wildfire_dynamics_southeastafrica_outputs.zip'

with zipfile.ZipFile(ZIP_NAME, 'w', zipfile.ZIP_DEFLATED) as zf:
    # Figures
    for root, _, files in os.walk('outputs'):
        for file in files:
            zf.write(os.path.join(root, file))
    # Processed CSVs
    for csv in [
        'data/processed/monthly_burned_area_2015_2024.csv',
        'data/processed/country_burned_area.csv',
        'data/processed/s5p_co_country_2018_2024.csv',
        'data/processed/s5p_no2_country_2018_2024.csv',
        'data/processed/s5p_aer_ai_country_2018_2024.csv',
        'data/processed/forest_cover_loss_country_2015_2023.csv',
        'data/processed/fire_by_landcover.csv',
    ]:
        if os.path.exists(csv):
            zf.write(csv)

print(f'Outputs packaged: {ZIP_NAME}')

from google.colab import files
files.download(ZIP_NAME)
